In [1]:
import sys
## save variables
import pickle
## folder names
from glob import glob
## wav import
from scipy.io import wavfile 
## standard libraries
import numpy as np
## MFCC
#!{sys.executable} -m pip install msgpack --user
#!{sys.executable} -m pip install python_speech_features --user

from python_speech_features import mfcc
from python_speech_features import logfbank
from python_speech_features import delta

import random as rnd
import os.path
import tarfile

from six.moves import urllib

import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib notebook
from scipy import signal
from scipy.io import wavfile


#!{sys.executable} -m pip install opencv-python --user
#!{sys.executable} -m pip install opencv-contrib-python --user
#import cv2
#garbage collector
import gc
#OS detection
import platform

#!{sys.executable} -m pip install librosa --user
import librosa

#!{sys.executable} -m pip install progressbar2 --user
import progressbar

from dependencies import functions
import functools

C:\Users\yurin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#make variables directory if not present
dest_directory = 'variables/'
if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)

        #data url from which download the dataset      
data_url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'

#make dataset directory if not present
dest_directory = 'dataset/'
if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)

#select the last part of the dataurl (the file name)      
filename = data_url.split('/')[-1]
filepath = os.path.join(dest_directory, filename)

#program the download and extraction if the file doesn't exists
if not os.path.exists(filepath):
    def progress(count, block_size, total_size):
        sys.stdout.write(
            '\r>> Downloading %s %.1f%%' %
            (filename, float(count * block_size) / float(total_size) * 100.0))
        sys.stdout.flush()
    try:
        filepath, _ = urllib.request.urlretrieve(data_url, filepath, progress)
    except:
        print(Error)
        
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [3]:
coreKey = ['yes', "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero",
           "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
sampleRate = 16000

In [23]:
#control if the raw data are saved with pickle
#if true they will be loaded in rawDict
reDo = True
pathName = {}
if os.path.exists('variables/rawDict.pkl') and not reDo:
    print('rawDict found')
    with open('variables/rawDict.pkl', 'rb') as f:  
        rawDict = pickle.load(f)
    with open('variables/pathName.pkl', 'rb') as f:  
        pathName = pickle.load(f)
    print('rawDict loaded')

#else they will be loaded from wav files
else:
    print('Creating rawDict')
    folders = glob(os.path.join('dataset', '*', ''))
    folders.remove(os.path.join('dataset', '_background_noise_', ''))
    print('SIGNALS')
    rawDict = {}
    for key in folders:
        print('Processing ', key)
        dictKey = os.path.basename(os.path.dirname(key))
        tmpFiles = glob(key+'*')
        array = []
        pathList = []
        for file in tmpFiles:
            pathList.append(file)
            tmp = wavfile.read(file)[1].copy()
            tmp.resize(16000, refcheck=False)
            array.append(tmp)
        rawDict[dictKey] = np.array(array)
        pathName[dictKey] = pathList
    #and saved with pickle
    with open('variables/rawDict.pkl', 'wb') as f:  
        pickle.dump(rawDict, f)
    with open('variables/pathName.pkl', 'wb') as f:  
        pickle.dump(pathName, f)
    print('rawDict created and saved to variables/rawDict.pkl')
reDo = True
#the same with noise signals
if not os.path.exists('variables/noiseDict.pkl') or reDo:    
    print('\nNOISE')
    noiseDict = {}
    folders = glob(os.path.join('dataset', '_background_noise_', '*.wav'))
    for key in folders:
        print('Processing ', key)
        noiseDict[os.path.basename(key)] = wavfile.read(key)[1]
    with open('variables/noiseDict.pkl', 'wb') as f:  
        pickle.dump(noiseDict, f) 
    print('noiseDict created and saved to variables/noiseDict.pkl')
else:
    print('noiseDict found')
    with open('variables/noiseDict.pkl', 'rb') as f:  
        noiseDict = pickle.load(f)
    print('noiseDict loaded')

Creating rawDict
SIGNALS
Processing  dataset\backward\
Processing  dataset\bed\
Processing  dataset\bird\
Processing  dataset\cat\
Processing  dataset\dog\
Processing  dataset\down\
Processing  dataset\eight\
Processing  dataset\five\
Processing  dataset\follow\
Processing  dataset\forward\
Processing  dataset\four\
Processing  dataset\go\
Processing  dataset\happy\
Processing  dataset\house\
Processing  dataset\learn\
Processing  dataset\left\
Processing  dataset\marvin\
Processing  dataset\nine\
Processing  dataset\no\
Processing  dataset\off\
Processing  dataset\on\
Processing  dataset\one\
Processing  dataset\right\
Processing  dataset\seven\
Processing  dataset\sheila\
Processing  dataset\six\
Processing  dataset\stop\
Processing  dataset\three\
Processing  dataset\tree\
Processing  dataset\two\
Processing  dataset\up\
Processing  dataset\visual\
Processing  dataset\wow\
Processing  dataset\yes\
Processing  dataset\zero\
rawDict created and saved to variables/rawDict.pkl

NOISE
Pr

C:\Users\yurin\Anaconda3\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Preprocessing

## Shift

In [25]:
%%time
shift_percentage = 0.1
time_shift_max = 100 #[ms]
sample_shift_max = round(time_shift_max / 1000 * sampleRate)

for key in rawDict:
    length = rawDict[key].shape[0]
    toShift = round(length*shift_percentage)
    for i in rnd.sample(range(length),toShift):
        shift = rnd.randint(-sample_shift_max, sample_shift_max)
        rawDict[key][i] = functions.shiftVec(rawDict[key][i],shift)        

Wall time: 905 ms


## Noise

In [26]:
%%time
noise_percentage = 0.1 
max_noise = .5
for key in rawDict:
    length = rawDict[key].shape[0]
    toNoise = round(length*noise_percentage)
    for i in rnd.sample(range(length),toNoise):
        noise = functions.noiseSelector(noiseDict, sampleRate)
        rawDict[key][i] += np.array(np.round(np.random.uniform(high = max_noise) * noise),dtype='int16')

Wall time: 1.47 s


## Silence creation

In [27]:
%%time
key = 'silence'
silence_percentage = 0.05
max_noise_sound = 0.5
#silence_max = round(tot_samples*silence_percentage)
silence_max = 5000
values = []
for i in range(silence_max):
    noise = functions.noiseSelector(noiseDict, sampleRate)
    sig = noise * np.random.uniform(high = max_noise_sound)
    values.append(sig)
values = np.array(values)
rawDict[key]=values

Wall time: 639 ms


## Downsampling

In [28]:
%%time
from multiprocessing.dummy import Pool 
def resample(sample, sr = 16000, n_sr = 8000, res_type = 'kaiser_fast', scale = True):
    return librosa.core.resample(np.array(sample, dtype = 'float32'), sr, n_sr, res_type = res_type, scale = scale)
pool = Pool(4)

print('Downsampling rawDict')
for count, key in enumerate(rawDict):
    print('Processing ',key, " (", count+1, "/", len(rawDict),")" )
    resampledKey = []
    a = list(rawDict[key])    
    results = pool.map(resample, a)
    results = np.array(results, dtype='int16')
    rawDict[key] = results
    
print('Downsampling noiseDict')
for count, key in enumerate(noiseDict):
    print('Processing ',key, " (", count+1, "/", len(noiseDict),")" )
    resampledKey = []   
    results = np.array(resample(noiseDict[key]), dtype='int16')
    noiseDict[key] = results
sampleRate = int(sampleRate/2)

Processing  backward  ( 1 / 36 )
Processing  bed  ( 2 / 36 )
Processing  bird  ( 3 / 36 )
Processing  cat  ( 4 / 36 )
Processing  dog  ( 5 / 36 )
Processing  down  ( 6 / 36 )
Processing  eight  ( 7 / 36 )
Processing  five  ( 8 / 36 )
Processing  follow  ( 9 / 36 )
Processing  forward  ( 10 / 36 )
Processing  four  ( 11 / 36 )
Processing  go  ( 12 / 36 )
Processing  happy  ( 13 / 36 )
Processing  house  ( 14 / 36 )
Processing  learn  ( 15 / 36 )
Processing  left  ( 16 / 36 )
Processing  marvin  ( 17 / 36 )
Processing  nine  ( 18 / 36 )
Processing  no  ( 19 / 36 )
Processing  off  ( 20 / 36 )
Processing  on  ( 21 / 36 )
Processing  one  ( 22 / 36 )
Processing  right  ( 23 / 36 )
Processing  seven  ( 24 / 36 )
Processing  sheila  ( 25 / 36 )
Processing  six  ( 26 / 36 )
Processing  stop  ( 27 / 36 )
Processing  three  ( 28 / 36 )
Processing  tree  ( 29 / 36 )
Processing  two  ( 30 / 36 )
Processing  up  ( 31 / 36 )
Processing  visual  ( 32 / 36 )
Processing  wow  ( 33 / 36 )
Processing  y

## Data division

In [29]:
%%time
trainDict = {}
validationDict = {}
testDict = {}

validation_percentage = 10
testing_percentage = 10

for key in rawDict:
    if key == 'silence':
        #already random so 
        trainDict[key]= rawDict[key][0:round(silence_max*(1-validation_percentage/100-testing_percentage/100))]
        validationDict[key] = rawDict[key][round(silence_max*(1-validation_percentage/100-testing_percentage/100)):round(silence_max*(1-testing_percentage/100))]
        testDict[key] = rawDict[key][round(silence_max*(1-testing_percentage/100)):silence_max]
    else:
        testTemp = []
        trainTemp = []
        validTemp = []
        for count, sample in enumerate(rawDict[key]):
            assign = functions.which_set(pathName[key][count], validation_percentage, testing_percentage, 2**27 - 1)
            if assign == 'testing':
                testTemp.append(sample)
            elif assign == 'training':
                trainTemp.append(sample)
            elif assign == 'validation':
                validTemp.append(sample)
        trainDict[key]= np.array(trainTemp)
        validationDict[key] = np.array(validTemp)
        testDict[key] = np.array(testTemp)

Wall time: 1.44 s


## Data augmentation
Only on core keys and on train set

In [30]:
def getStretch():
    low = np.round(np.random.uniform(low = 0.8, high = 0.9), decimals = 2)
    high = np.round(np.random.uniform(low = 1.1, high = 1.2), decimals = 2)
    return rnd.choice([low,high])

In [31]:
%%time
import progressbar

rnd.seed(1)

augmented_percentage = 0.8

def newSample(sample, max_noise = .2, sampleRate = 8000, time_shift_max = 50):
    sample_shift_max = round(time_shift_max / 1000 * sampleRate)
    shift = rnd.randint(-sample_shift_max, sample_shift_max)
    new_sample = functions.shiftVec(sample,shift)     
    noise = functions.noiseSelector(noiseDict, sampleRate)
    new_sample += np.array(np.round(np.random.uniform(high = max_noise) * noise),dtype='int16')
    new_sample = functions.stretch(np.array(new_sample,dtype='float32') , getStretch())
    new_sample = np.array(librosa.effects.pitch_shift(new_sample, sampleRate, n_steps = rnd.choice([-3.5, -2.5, -2, -1, 1, 2, 2.5, 3.5])), dtype = 'int16')
    return sample
    
pool = Pool(2)
trainDictAugmented = {}
for countKey, key in enumerate(coreKey):
    length = trainDict[key].shape[0]
    toAugment = round(length*augmented_percentage)
    new_bunch_of_samples = []
    iterate = rnd.sample(range(length),toAugment)
    widgets = [
        'Processing ' +str(key)+ " ("+ str(countKey+1) + "/" + str(len(coreKey)) + ")",
        '  ', progressbar.ETA()
        
    ]
    bar = progressbar.ProgressBar(widgets=widgets, max_value=len(iterate))
    bar.start()
    samples = trainDict[key][iterate]
    results = np.array(pool.map(newSample, samples))
    bar.finish()
    trainDictAugmented[key] = results

key = 'silence'
max_noise_sound = 0.5
#silence_max = round(tot_samples*silence_percentage)
silence_max = 3000
values = []
for i in range(silence_max):
    noise = functions.noiseSelector(noiseDict, sampleRate)
    sig = noise * np.random.uniform(high = max_noise_sound)
    values.append(sig)
values = np.array(values)
trainDictAugmented[key]=values

Processing yes (1/20)  Time:  0:00:31                                          
Processing no (2/20)  Time:  0:00:29                                           
Processing up (3/20)  Time:  0:00:28                                           
Processing down (4/20)  Time:  0:00:29                                         
Processing left (5/20)  Time:  0:00:28                                         
Processing right (6/20)  Time:  0:00:28                                        
Processing on (7/20)  Time:  0:00:29                                           
Processing off (8/20)  Time:  0:00:27                                          
Processing stop (9/20)  Time:  0:00:29                                         
Processing go (10/20)  Time:  0:00:29                                          
Processing zero (11/20)  Time:  0:00:30                                        
Processing one (12/20)  Time:  0:00:29                                         
Processing two (13/20)  Time:  0:00:29  

Wall time: 9min 48s


# Feature extraction

## MFCC

In [32]:
# list with mfcc parameters in order: [numcep, nfilt, winlen, winstep]+
values = [[14,26,0.025,0.01,512]]

\begin{verbatim}
Parameter         Description
signal 	          the audio signal from which to compute features. Should be an N*1 array
samplerate 	      the samplerate of the signal we are working with.
winlen 	          the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
winstep 	      the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
numcep 	          the number of cepstrum to return, default 13
nfilt 	          the number of filters in the filterbank, default 26.
nfft 	          the FFT size. Default is 512
lowfreq 	      lowest band edge of mel filters. In Hz, default is 0
highfreq 	      highest band edge of mel filters. In Hz, default is samplerate/2\\
preemph 	      apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97\\
ceplifter 	      apply a lifter to final cepstral coefficients. 0 is no lifter. Default is 22\\
appendEnergy 	  if this is true, the zeroth cepstral coefficient is replaced with the log of the total frame energy.\\
returns 	      A numpy array of size (NUMFRAMES by numcep) containing features. Each row holds 1 feature vector.\\
\end{verbatim}

In [33]:
def computeMFCC(data, output, i, sampleRate = 8000):
    for sig in data:
        mfcc_feat = mfcc(sig,sampleRate, preemph = 0.97, numcep = i[0], nfilt = i[1], winlen = i[2], winstep = i[3], nfft = i[4], lowfreq=100, highfreq=4000)
        output.append(mfcc_feat)
def computeLogF(data, output, i, sampleRate = 8000):
    for sig in data:
        lfilt_feat = logfbank(sig,sampleRate, preemph = 0.97, nfilt = i[1], winlen = i[2], winstep = i[3], nfft = i[4], lowfreq=100, highfreq=4000)
        output.append(lfilt_feat)
def computeDelta(mfccValues, N = 2):
    temp = []
    for count, _ in enumerate(mfccValues):
        delt = delta(mfccValues[count], N)
        temp.append(delt)
    return temp        

In [34]:
%%time

def getName(index):
    if index == 0:
        return 'Train'
    elif index == 1:
        return 'Test'
    elif index == 2:
        return 'Validation'
    elif index == 3:
        return 'AugmentedTrain'
    
#look for the already computed mfcc
dictionaries = [trainDict, testDict, validationDict, trainDictAugmented]
for count, i in enumerate(values):
    for index, dictionary in enumerate(dictionaries):
        print('\n#####Coumputing '+getName(index)+ ' Set#####')
        nameMFCC = 'variables/mfccDict'+functions.getName(index)+'[nC='+str(i[0])+' wL='+str(i[2])+' wS='+str(i[3])+'].pkl'
        nameDelta = 'variables/mfccDictDD'+functions.getName(index)+'[nC='+str(i[0])+' wL='+str(i[2])+' wS='+str(i[3])+'].pkl'
        mfccDict = {}
        for countKey, key in enumerate(dictionary):
            print('Processing ',key, " (", countKey+1, "/", len(dictionary),")" )
            array = []
            computeMFCC(dictionary[key], array, i, sampleRate = sampleRate)            
            mfccDict[key] = np.array(array)
        
        with open(nameMFCC, 'wb') as f:  
            pickle.dump(mfccDict, f)
            
        print("\n  Processing delta and delta-delta")
        for countKey, key in enumerate(mfccDict):
            print('Processing ',key, " (", countKey+1, "/", len(mfccDict),")" )
            delt = np.array(computeDelta(mfccDict[key]))
            deltdelt = np.array(computeDelta(delt))
            mfccDict[key] = np.stack([mfccDict[key],delt,deltdelt], axis = -1)
        
        with open(nameDelta, 'wb') as f:  
            pickle.dump(mfccDict, f)


#####Coumputing Train Set#####
Processing  backward  ( 1 / 36 )
Processing  bed  ( 2 / 36 )
Processing  bird  ( 3 / 36 )
Processing  cat  ( 4 / 36 )
Processing  dog  ( 5 / 36 )
Processing  down  ( 6 / 36 )
Processing  eight  ( 7 / 36 )
Processing  five  ( 8 / 36 )
Processing  follow  ( 9 / 36 )
Processing  forward  ( 10 / 36 )
Processing  four  ( 11 / 36 )
Processing  go  ( 12 / 36 )
Processing  happy  ( 13 / 36 )
Processing  house  ( 14 / 36 )
Processing  learn  ( 15 / 36 )
Processing  left  ( 16 / 36 )
Processing  marvin  ( 17 / 36 )
Processing  nine  ( 18 / 36 )
Processing  no  ( 19 / 36 )
Processing  off  ( 20 / 36 )
Processing  on  ( 21 / 36 )
Processing  one  ( 22 / 36 )
Processing  right  ( 23 / 36 )
Processing  seven  ( 24 / 36 )
Processing  sheila  ( 25 / 36 )
Processing  six  ( 26 / 36 )
Processing  stop  ( 27 / 36 )
Processing  three  ( 28 / 36 )
Processing  tree  ( 29 / 36 )
Processing  two  ( 30 / 36 )
Processing  up  ( 31 / 36 )
Processing  visual  ( 32 / 36 )
Processing

# LogFilter with delta


In [35]:
%%time
for count, i in enumerate(values):
    for index, dictionary in enumerate(dictionaries):
        print('\n#####Coumputing '+functions.getName(index)+ ' Set#####')
        nameLog = 'variables/logfiltDict'+functions.getName(index)+'[nF='+str(i[1])+' wL='+str(i[2])+' wS='+str(i[3])+'].pkl'
        nameLogDD = 'variables/logfiltDictDD'+functions.getName(index)+'[nF='+str(i[1])+' wL='+str(i[2])+' wS='+str(i[3])+'].pkl'
        logFDict = {}
        for countKey, key in enumerate(dictionary):
            print('Processing ',key, " (", countKey+1, "/", len(rawDict),")" )
            array = []
            computeLogF(dictionary[key], array, i, sampleRate = sampleRate)            
            logFDict[key] = np.array(array)
        
        with open(nameLog, 'wb') as f:  
            pickle.dump(logFDict, f)
            
        print("\n  Processing delta and delta-delta")
        
        for countKey, key in enumerate(logFDict):
            print('Processing ',key, " (", countKey+1, "/", len(logFDict),")" )
            delt = np.array(computeDelta(logFDict[key]))
            deltdelt = np.array(computeDelta(delt))
            logFDict[key] = np.stack([logFDict[key],delt,deltdelt], axis = -1)
        
        with open(nameLogDD, 'wb') as f:  
            pickle.dump(logFDict, f)


#####Coumputing Train Set#####
Processing  backward  ( 1 / 36 )
Processing  bed  ( 2 / 36 )
Processing  bird  ( 3 / 36 )
Processing  cat  ( 4 / 36 )
Processing  dog  ( 5 / 36 )
Processing  down  ( 6 / 36 )
Processing  eight  ( 7 / 36 )
Processing  five  ( 8 / 36 )
Processing  follow  ( 9 / 36 )
Processing  forward  ( 10 / 36 )
Processing  four  ( 11 / 36 )
Processing  go  ( 12 / 36 )
Processing  happy  ( 13 / 36 )
Processing  house  ( 14 / 36 )
Processing  learn  ( 15 / 36 )
Processing  left  ( 16 / 36 )
Processing  marvin  ( 17 / 36 )
Processing  nine  ( 18 / 36 )
Processing  no  ( 19 / 36 )
Processing  off  ( 20 / 36 )
Processing  on  ( 21 / 36 )
Processing  one  ( 22 / 36 )
Processing  right  ( 23 / 36 )
Processing  seven  ( 24 / 36 )
Processing  sheila  ( 25 / 36 )
Processing  six  ( 26 / 36 )
Processing  stop  ( 27 / 36 )
Processing  three  ( 28 / 36 )
Processing  tree  ( 29 / 36 )
Processing  two  ( 30 / 36 )
Processing  up  ( 31 / 36 )
Processing  visual  ( 32 / 36 )
Processing